In [1]:
from pathlib import Path
from bs4 import BeautifulSoup

In [2]:
from IPython.core.display import display, HTML
import re
import numpy as np

def get_question_text(question_soup):
    return question_soup.find_all('text')[0].contents[0]

def replace_base_uri(sessieXML, question_text):
    return question_text.replace('__BASE_URI__', f'{sessieXML.parent / "web_folders"}/')

def remove_answer_fields(question_text):
    return re.sub(r'.*\<\d+\>.*', '', question_text)

def get_question_html(sessieXML, question_soup):
    q_text = get_question_text(question_soup)
    q_text = replace_base_uri(sessieXML, q_text)
    q_text = remove_answer_fields(q_text)
    return q_text

def get_session_name(soup):
    return str(soup.module.find('name').contents[0]).strip()

def get_session_quesions(soup):
    return soup.find_all('question')

def get_quesion_name(q_soup):
    return str(q_soup.find('name').contents[0]).strip()

def get_session_html(sessieXML, soup):
    session_name = get_session_name(soup)
    questions = get_session_quesions(soup)
    question_names = [
        get_quesion_name(q_soup)
        for q_soup in questions
    ]
    
    idxs = np.argsort(question_names)

    session_html = f'<h1>{session_name}</h1>\n'

    for i, idx in enumerate(idxs):
        q_soup = questions[idx]
        q_name = question_names[idx]
        session_html += f'<h2>Oefening {i + 1} ({q_name})</h2>\n'
        session_html += get_question_html(sessieXML, q_soup)
        
    return session_html

In [3]:
import os

def create_sessions_docx(path):
    all_sessions_html = ''

    for sessieXML in sorted(list(path.glob('*/manifest.xml'))):
        soup = BeautifulSoup(sessieXML.read_text(), "html.parser")
        all_sessions_html += get_session_html(sessieXML, soup)

    out_path = Path(f'{path.stem}.html')
    out_path.write_text(all_sessions_html)
    os.system(f'pandoc {out_path} -f html -t docx -s -o {out_path.parent / out_path.stem}.docx')

    #display(HTML(session_html))

In [4]:
create_sessions_docx(Path('./oefensessies/'))

In [5]:
create_sessions_docx(Path('./extra_oefensessies/'))